# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

- Intermediate-level knowledge of Python 3 (NumPy and Pandas preferably, but not required)
- Exposure to PyTorch usage
- Basic understanding of Deep Learning and Language Models (BERT specifically)

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.pdf" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv(
    'smile-annotations-final.csv', 
    names=['id','text','category'])
df.set_index('id',inplace=True)

In [3]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [4]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [5]:
df = df[(~df['category'].str.contains(r"\|", regex=True))\
        &(~df['category'].str.contains("nocode"))]

In [6]:
df

,text,category
id,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy
...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy


In [7]:
emotion2idx = {emote:i for i,emote in enumerate(df.category.unique())}

In [8]:
df['labels'] = df.category.map(emotion2idx)

## Task 3: Training/Validation Split

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.labels.values,
    test_size=0.15,
    random_state=17,
    stratify=df.labels.values
)

In [11]:
df['data_type'] = ['not_set']*df.shape[0]

In [12]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [13]:
df.groupby(['category','labels','data_type']).count()

text
category     labels data_type      
angry        2      train        48
                    val           9
disgust      3      train         5
                    val           1
happy        0      train       966
                    val         171
not-relevant 1      train       182
                    val          32
sad          4      train        27
                    val           5
surprise     5      train        30
                    val           5

## Task 4: Loading Tokenizer and Encoding our Data

In [14]:
!pip install transformers

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 13.0MB/s 
     |████████████████████████████████| 1.1MB 31.4MB/s 
     |████████████████████████████████| 890kB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b9ad5f032cceabff4136583a1952877441d784fe5eef3a5141fb0bd4c71f2503
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [15]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [16]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_len=256,
    return_tensors='pt'
)
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_len=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].labels.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].labels.values)

In [18]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [19]:
len(dataset_train),len(dataset_val)

(1258, 223)

## Task 5: Setting up BERT Pretrained Model

In [20]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(emotion2idx),
    output_attentions=False,
    output_hidden_states=False
)

## Task 6: Creating Data Loaders

In [22]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [23]:
batch_size = 32
dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)


## Task 7: Setting Up Optimizer and Scheduler

In [24]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [25]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

In [26]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [27]:
import numpy as np

In [28]:
from sklearn.metrics import f1_score

In [29]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(preds_flat,labels_flat,average='weighted')

In [44]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in emotion2idx.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])/len(y_true)}')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [31]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [32]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [37]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [36]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b  in batch)
        
        inputs = {
            'input_ids':batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]
        }
        
        output = model(**inputs)
        
        loss = output[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'train_loss':'{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'BERT_ft_epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total / len(dataloader_train)
    
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')


Epoch 1
Training loss: 0.4569181356165144
Validation loss: 0.6230950823851994
F1 score (weighted): 0.8686605475592624



Epoch 2
Training loss: 0.3228747561810509
Validation loss: 0.6329664226089206
F1 score (weighted): 0.8942605086219113



Epoch 3
Training loss: 0.19393239007109686
Validation loss: 0.6880502764667783
F1 score (weighted): 0.8927347478340789



Epoch 4
Training loss: 0.12494902431018769
Validation loss: 0.7155227384396962
F1 score (weighted): 0.8545358607848037



Epoch 5
Training loss: 0.08228844214999487
Validation loss: 0.7233642935752869
F1 score (weighted): 0.8977524883734532



Epoch 6
Training loss: 0.06167335188578046
Validation loss: 0.7154904689107623
F1 score (weighted): 0.8752487690776806



Epoch 7
Training loss: 0.04325037447233049
Validation loss: 0.7416209280490875
F1 score (weighted): 0.895534379671151



Epoch 8
Training loss: 0.03328285188901992
Validation loss: 0.7124350837298802
F1 score (weighted): 0.8802831925934739



Epoch 9
Training loss: 0.028209752317458866
Validation loss: 0.7233010871069772
F1 score (weighted): 0.895507144485825



Epoch 10
Training loss: 0.02606615537688846
Validation loss: 0.7231176027229854
F1 score (weighted): 0.895507144485825



## Task 10: Loading and Evaluating our Model

In [38]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(emotion2idx),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [39]:
model = model.to(device)

In [41]:
model.load_state_dict(torch.load('BERT_ft_epoch1.model',
                      map_location=torch.device('cuda:0')))

<All keys matched successfully>

In [42]:
_,predictions, true_vals = evaluate(dataloader_val)

In [45]:
accuracy_per_class(predictions,true_vals)

Class: happy
Accuracy: 0.9707602339181286
Class: not-relevant
Accuracy: 0.59375
Class: angry
Accuracy: 0.2222222222222222
Class: disgust
Accuracy: 0.0
Class: sad
Accuracy: 0.0
Class: surprise
Accuracy: 0.0
